---
# Stock Project
---

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pymysql, calendar, time, json
import requests
from datetime import datetime
from threading import Timer

# 데이터 수집

In [2]:
# class DBUpdater:
#     def __init__(self):         # 클래스가 호출되면 자동 실행되는 함수
#         print('시작')
    
#     def __del__(self):          # 클래스가 종료될 때 자동 실행되는 함수
#         print('종료')
    
#     # 내가 했던 것    
#     def read_krx_code(self):    # krx 데이터를 불러오는 함수
#         url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
#         tables = pd.read_html(url)[0]       # 데이터 크롤링 -> 원래는 크롤링 코드인데 지금 홈페이지 안 열려서 이런식으로 가져오기
#         tables.rename(columns={'종목코드':'code','회사명':'company'},inplace=True)      # 칼럼 rename
#         tables['code'] = tables['code'].apply(lambda x: str(x).zfill(6))        # 종목코드 6자리에 맞춰 0으로 패딩
#         return tables
    

In [3]:
# dbu = DBUpdater()

In [4]:
# tables = dbu.read_krx_code()

## DB에 데이터 추가

In [5]:
# class DBUpdater:
#     def __init__(self):         # 클래스가 호출되면 자동 실행되는 함수
#         print('시작')
    
#     def __del__(self):          # 클래스가 종료될 때 자동 실행되는 함수
#         print('종료')
    
#     # 교수님이랑 같이 한 것    
#     def read_krx_code(self):    # krx 데이터를 불러오는 함수
#         url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
#         krx = pd.read_html(url,header=0)[0]       # 데이터 크롤링 -> 원래는 크롤링 코드인데 지금 홈페이지 안 열려서 이런식으로 가져오기
#         krx = krx[['종목코드','회사명']]
#         krx.rename(columns={'종목코드':'code','회사명':'company'},inplace=True)      # 칼럼 rename
#         # krx['code'] = krx['code'].apply(lambda x: str(x).zfill(6))        # 종목코드 6자리에 맞춰 0으로 패딩
#         krx.code = krx.code.map('{:06d}'.format)                            # 종목코드 6자리에 맞춰 0으로 패딩 - 교수님 버전
#         return krx
    
#     def db_insert(self,df):
#         try:
#             conn = pymysql.connect(host = 'localhost',user = 'root',password = 'apple123!!',db = 'investar',charset='utf8')
#             cur = conn.cursor()
#             cur.execute("create table if not exists krx (company char(20), code char(6));")

#             sql = 'INSERT INTO krx (company, code) VALUES (%s, %s)'
#             values = [tuple(row) for row in df[['company', 'code']].values]

#             cur.executemany(sql, values)
#             conn.commit()

#         finally:
#             cur.close()
#             conn.close()

In [6]:
# tables.head()

### 교수님

In [7]:
class DBUpdater:
    # def __init__(self):         # 클래스가 호출되면 자동 실행되는 함수
    #     self.conn = pymysql.connect(host='localhost',user = 'root',password = 'apple123!!',db = 'investar',charset='utf8')
        
    #     with self.conn.cursor() as curs:
    #         sql = """
    #         create table if not exists company_info(
    #             code varchar(20),
    #             company varchar(40),
    #             last_update DATE,
    #             PRIMARY KEY (code));
    #         """
                
    #         curs.execute(sql)
            
    #         sql = """
    #         CREATE TABLE IF NOT EXISTS daily_price(
    #             code VARCHAR(20),
    #             date DATE
    #             open BIGINT(20),
    #             high BIGINT(20),
    #             low BIGINT(20),
    #             closs BIGINT(20),
    #             diff BIGINT(20),
    #             value BIGINT(20),
    #             PRIMARY KEY (code, date));
    #         """
    #         curs.execute(sql)
            
    #     self.conn.commit()
    #     self.codes = dict()     


    def __init__(self):
        self.conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db='investar', charset='utf8')

        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info (
                code VARCHAR(20),
                company VARCHAR(40),
                last_update DATE,
                PRIMARY KEY (code)
            );
            """
            curs.execute(sql)

            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                value BIGINT(20),
                PRIMARY KEY (code, date)
            );
            """
            curs.execute(sql)

        self.conn.commit()
        self.codes = dict()
    
    def __del__(self):          # 클래스가 종료될 때 자동 실행되는 함수
        self.conn.close()
    
    # 교수님이랑 같이 한 것    
    def read_krx_code(self):    # krx 데이터를 불러오는 함수
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
        krx = pd.read_html(url,header=0)[0]       # 상장 기업 정보
        krx = krx[['종목코드','회사명']]
        krx.rename(columns={'종목코드':'code','회사명':'company'},inplace=True)     # 칼럼 rename
        krx.code = krx.code.map('{:06d}'.format)                                # 종목코드 6자리에 맞춰 0으로 패딩
        return krx
    
    def update_comp_info(self):
        # 1차 : DB에 있는 데이터 불러오기
        # DB에 있는 종목코드를 불러와 딕셔너리(추후 크롤링 키값으로 쓸 예정)에 저장
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]
        
        # 2차: DB에 있는 데이터가 최신 정보인지 판단
        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info"
            curs.execute(sql)
            rs = curs.fetchone()        # 최근 업데이트 일자 : DB 전체가 같이 업데이트 되기 때문에 하나만 긁어와도(조회) 됨 -> fetchone()
            
            # DB에 있는 데이터가 최신 정보가 아닐 때
            today = datetime.today().strftime('%Y-%m-%d')
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:     # 최근 업데이트 일자가 없거나 이전 데이터일 때
                
                # 데이터 업데이트
                krx = self.read_krx_code()                              # 종목 코드 다시 긁어오기
                for idx in range(len(krx)):                             # 코드 가져오기
                    code = krx.code.values[idx]                         # 회사 가져오기
                    company = krx.company.values[idx]
                    
                    # SQL DB 업데이트 -> replace 사용
                    sql = f"REPLACE INTO company_info (code, company, last_update) VALUES ('{code}', '{company}', '{today}')"
                    curs.execute(sql)
                    
                    # 딕셔너리 업데이트
                    self.codes[code]=company
                
                # 업데이트 시각 출력하기    
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info VALUES ({code}, {company}, {today})")
                
                    
                self.conn.commit()
    
    
    def read_naver(self, code, company, pages_to_fetch):
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")

            # 크롤링 범위 -> 가장 오래된 데이터를 찾기 위해 마지막 페이지 찾기
            pgrr = html.find("td", class_="pgRR")           # 마지막 페이지 이동 버튼
            if pgrr is None:
                return None

            last_page = int(pgrr.a["href"].split('=')[-1])  # 마지막 페이지 이동 버튼에서 실제 페이지 번호 가져오기
            pages = min(last_page, pages_to_fetch)          # 이용자가 선택한 페이지가 실제 마지막 페이지보다 많다면 오류가 뜨기 때문에 처리

            data_frames = []

            for page in range(1, pages + 1):
                page_url = f'{url}&page={page}'
                print(page_url)

                page_df = pd.read_html(requests.get(page_url, headers={'User-agent': 'Mozilla/5.0'}).text)[0]
                data_frames.append(page_df)

                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                print(f'[{tmnow}] {company} ({code}): {page:04d}/{pages:04d} pages are downloading...', end="\r")
            #     tmpDf = pd.read_html(requests.get(pg_url, headers={'User-agent':'Mozilla/5.0'}))
            # df = pd.concat((df, tmpDf),ignore_index=True)
            df = pd.concat(data_frames, ignore_index=True)
            df = df.rename(columns={'날짜': 'date', '종가': 'close', '전일비': 'diff',
                                    '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})
            df['date'] = df['date'].replace('.', '-')
            df = df.dropna()
            df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)
            df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]

        except requests.exceptions.RequestException as e:
            print('RequestException occurred:', str(e))
            return None
        except Exception as e:
            print('Exception occurred:', str(e))
            return None

        return df

    
    # 네이버에서 읽어온 주식 시세를 DB에 저장
    def replace_into_db(self, df, num, code, company):
        """네이버에서 읽어온 주식 시세를 DB에 저장"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price VALUES ('{code}', "\
                    f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, "\
                    f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            
            print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_'\
                'price [OK]'.format(datetime.now().strftime('%Y-%m-%d'\
                ' %H:%M'), num+1, company, code, len(df)))
     
     
    def update_daily_price(self, pages_to_fetch):
        for idx, code in enumerate(self.codes):             # self.codes에 들어있는 회사 전체 -> 이 부분을 고쳐서 특정 회사만 찾도록 조정 가능
            df = self.read_naver(code, self.codes[code], pages_to_fetch)
            if df is None:
                continue

            self.replace_into_db(df, idx, code, self.codes[code]) 
    
    # 매일 자동 실행하도록 만드는 함수     
    def execute_daily(self):
        self.update_comp_info()
        # 크롤링 페이지 컨트롤
        try:
            with open('config.json','r') as in_file:
                config = json.load(in_file)
                pages_to_fetch = config['pages_to_fetch']
        except FileNotFoundError:
            with open('config.json','w') as out_file:
                pages_to_fetch = 100
                config = {'pages_to_fetch' : pages_to_fetch}
                json.dump(config, out_file)
        
                
        # 실행 일시 세팅하기        
        tmnow = datetime.now()                                                          # 오늘 일자
        lastday = calendar.monthrange(tmnow.year, tmnow.month)[1]                       # 오늘 일자가 속한 월의 총 일수
        
        # 연도별/월별 마지막 날짜일 때 -> 1일로 리셋
        if tmnow.month == 12 and tmnow.day == lastday:                                                  # 연도 마지막 날
            tmnext = tmnow.replace(year=tmnow.year+1, month=1, day=1, hour=17, minute=0, second=0)
        elif tmnow.day == lastday:                                                                      # 월 마지막 날
            tmnext = tmnow.replace(month=tmnow.month+1, day=1, hour=17, minute=0, second=0)
        else:
            tmnext = tmnow.replace(day=tmnow.day+1, hour=17, second=0)                                  # 일반적으로 -> 내일 5시로 세팅
        self.update_daily_price(pages_to_fetch)      # update_daily_price를 정의했던 것에 따르면 pages_to_fetch를 2로 설정
        

        tmdiff = tmnext - tmnow
        secs = tmdiff.seconds
        
        t = Timer(secs, self.execute_daily)
        t.start()

In [8]:
# 이번달의 총 일수
tmnow = datetime.now()
lastday = calendar.monthrange(tmnow.year, tmnow.month)   # [1] 안 붙이면 튜플로 나와서 붙여줘야함
print(lastday,lastday[1],tmnow.day+20)
print(lastday[1] == tmnow.day +20)
print(lastday == tmnow.day)

(0, 31) 31 32
False
False


In [9]:
dbu = DBUpdater()

In [10]:
# krx = dbu.update_comp_info()
# df = dbu.read_naver('005930','삼성전자',700)

/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1312/71410188.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


[2024-01-12 09:52] #2667 REPLACE INTO company_info VALUES (331660, 한국미라클피플사, 2024-01-12)
http://finance.naver.com/item/sise_day.nhn?code=005930&page=1
http://finance.naver.com/item/sise_day.nhn?code=005930&page=2ding...
http://finance.naver.com/item/sise_day.nhn?code=005930&page=3ding...
http://finance.naver.com/item/sise_day.nhn?code=005930&page=4ding...
http://finance.naver.com/item/sise_day.nhn?code=005930&page=5ding...
http://finance.naver.com/item/sise_day.nhn?code=005930&page=6ding...
http://finance.naver.com/item/sise_day.nhn?code=005930&page=7ding...
http://finance.naver.com/item/sise_day.nhn?code=005930&page=8ding...
http://finance.naver.com/item/sise_day.nhn?code=005930&page=9ding...
http://finance.naver.com/item/sise_day.nhn?code=005930&page=10ing...
http://finance.naver.com/item/sise_day.nhn?code=005930&page=11ing...
http://finance.naver.com/item/sise_day.nhn?code=005930&page=12ing...
http://finance.naver.com/item/sise_day.nhn?code=005930&page=13ing...
http://finance.naver.

KeyboardInterrupt: 

In [12]:
dbu.execute_daily()

/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1312/71410188.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


http://finance.naver.com/item/sise_day.nhn?code=000020&page=1
http://finance.naver.com/item/sise_day.nhn?code=000020&page=2ding...
http://finance.naver.com/item/sise_day.nhn?code=000020&page=3ding...
http://finance.naver.com/item/sise_day.nhn?code=000020&page=4ding...
http://finance.naver.com/item/sise_day.nhn?code=000020&page=5ding...
[2024-01-12 10:21] #0001 동화약품 (000020) : 50 rows > REPLACE INTO daily_price [OK]
http://finance.naver.com/item/sise_day.nhn?code=000040&page=1
http://finance.naver.com/item/sise_day.nhn?code=000040&page=2ading...
http://finance.naver.com/item/sise_day.nhn?code=000040&page=3ading...
http://finance.naver.com/item/sise_day.nhn?code=000040&page=4ading...
http://finance.naver.com/item/sise_day.nhn?code=000040&page=5ading...
[2024-01-12 10:21] #0002 KR모터스 (000040) : 50 rows > REPLACE INTO daily_price [OK]
http://finance.naver.com/item/sise_day.nhn?code=000050&page=1
http://finance.naver.com/item/sise_day.nhn?code=000050&page=2ng...
http://finance.naver.com/ite